In [55]:
import re
from tqdm import tqdm
import gzip
import os
import regex
import numpy as np

In [58]:
def preparedata(lan, repeats, train=0):
    with gzip.open(f'./OSCAR-2109/packaged/{lan}/{lan}_part_1.txt.gz', 'rb') as f:
            text = f.readlines()
    type = 'test'
    for repeat in range(repeats):
        if repeat > repeats-train-1:
            type = 'train'
        file = open(f'./data/{lan}_{repeat}.{type}', 'w')
        i = 0
        skip = False
        for line in (text):
            if skip:
                break
            line = line.decode('utf-8')
            line = regex.sub(r'-[\p{L}_]', '', line)
            line = line[:-3].lower().replace(' ', '\x5f')
            line = [line[i:i+10] for i in range(0, len(line), 10)]
            for string in line:
                if len(string) == 10:
                    i += 1
                    if i < 100*(repeat):
                        continue
                    if i > 100*(repeat+1):
                        skip = True
                        break
                    file.write(f'{string}\n')

In [59]:
languages = ['ar', 'de', 'el', 'en', 'fr', 'hi', 'la', 'pl', 'ru', 'sw', 'zh']
for lan in tqdm(languages):
    preparedata(lan, 3)
preparedata('eo', 6, 3)

100%|██████████| 11/11 [00:44<00:00,  4.07s/it]


In [5]:
def makealphabet():
    with open('./utf8_sequence_0-0x10ffff_assigned_including-unprintable-asis.txt', 'rb') as f:
        text = f.readlines()

    writefile = open('./alphabet.txt', 'w') 

    for line in text:
        line = line.decode('utf-8')
        line = [line[i:i+10] for i in range(0, len(line), 10)]
        for string in line:
            writefile.write(f'{string}\n')

In [6]:
makealphabet()

In [62]:
languages = ['ar', 'de', 'el', 'en', 'eo', 'fr', 'hi', 'la', 'pl', 'ru', 'sw', 'zh']
transl = ['ar', 'el', 'hi', 'ru','zh']
for lan in tqdm(transl):
    abet = np.array([])

    for r in range(3):
        with open(f'./data/{lan}_transl_{r}.test', 'r') as f:
            text = f.readlines()

        with open(f'./data/eo_{r}.train', 'r') as f:
            eo = f.readlines()

        for line in text:
            chars = list(line)
            abet = np.append(abet, chars)
            abet = np.unique(abet)
        
        for line in eo:
            chars = list(line)
            abet = np.append(abet, chars)
            abet = np.unique(abet)

    with open(f'./alphabets/alphabet_{lan}_transl.txt', 'w') as f:
        for char in abet:
            f.write(char)

100%|██████████| 5/5 [00:00<00:00,  5.82it/s]


In [27]:
print(len(abet))

315


In [28]:
with open('./alphabet.txt', 'w') as f:
    for char in abet:
        f.write(char)

#### Preparing of transliterated langs

In [60]:
def preparetransl(lan, repeats, train=0):
    with open(f'./data/{lan}_transl.test', 'rb') as f:
            text = f.readlines()
    type = 'test'
    for repeat in range(repeats):
        if repeat > repeats-train-1:
            type = 'train'
        file = open(f'./data/{lan}_transl_{repeat}.{type}', 'w')
        i = 0
        skip = False
        for line in (text):
            if skip:
                break
            line = line.decode('utf-8')
            line = regex.sub(r'-[\p{L}_]', '', line)
            line = line[:-3].lower().replace(' ', '\x5f')
            line = [line[i:i+10] for i in range(0, len(line), 10)]
            for string in line:
                if len(string) == 10:
                    i += 1
                    if i < 100*(repeat):
                        continue
                    if i > 100*(repeat+1):
                        skip = True
                        break
                    file.write(f'{string}\n')

In [61]:
for lan in ['ar', 'el', 'hi', 'ru', 'zh']:
    preparetransl(lan, 3)